# Java 9 — Vlerësimi (Evaluation)

**Objektivat:**
- Të ekzekutohet benchmark-u 40 pyetjesh.
- Të maten: saktësia, grounding, refuzimet.
- Të bëhet analizë gabimesh.

**Detyra (Assignment):**
- Ngarko `evaluation_benchmark_v2_complete.json`.
- Kalo të gjitha pyetjet në sistemin tuaj RAG/agent.
- Llogarit metrika: EM/partial, refusal correctness, citation presence.

---

# Doctrine & SOP Assistant (Fiktiv) — Google Colab Notebook

**Kujdes / Safety (e detyrueshme):**
- Ky projekt është *arsimor* dhe përdor dokumente **fiktive, të paklasifikuara**.
- Asistenti **nuk** jep këshilla taktike/operacionale.
- Nëse informacioni nuk gjendet në dokumente → **refuzim i sigurt** (“Nuk mbulohet nga dokumentet e disponueshme.”).

---

## 📁 Struktura e të dhënave (rekomandim)

Vendosni PDF-të në:
- `data/raw_pdfs/`

Emrat e rekomanduar (shembull):
- `AAF-TRN-001_SOP_e_Stërvitjes.pdf`
- `AAF-SAFE-010_Rregullat_e_Sigurisë_në_Poligon.pdf`
- `AAF-LOG-004_Siguria_dhe_Kontrolli_i_Mjeteve.pdf`
- `AAF-CMD-007_Zinxhiri_Komandues_dhe_Raportimi_i_Incidenteve.pdf`
- `AAF-GRD-002_Procedurat_e_Sherbimit_te_Rojes.pdf`
- `NATO_Permbledhje_Doktrinore_Edukative.pdf`
- `AAF-IDX-000_Indeksi_Doktrinor_dhe_Procedural.pdf`
- `AAF-ADM-012_Rregulla_Administrative_dhe_Arkivimi.pdf`
- `AAF-RPT-009_Udhezime_te_Pergjithshme_per_Raportim.pdf`


## 0) Setup & Import

In [ ]:
!pip -q install sentence-transformers chromadb pypdf faiss-cpu rapidfuzz

import os, re, json, math
from pathlib import Path


## 1) Ngarko PDF-të (zgjidh një mënyrë)

### Opsioni A — Upload direkt në Colab

In [ ]:
from google.colab import files
uploaded = files.upload()  # zgjidh PDF-të nga kompjuteri
os.makedirs("data/raw_pdfs", exist_ok=True)
for fn in uploaded.keys():
    Path(fn).replace(Path("data/raw_pdfs")/fn)
print("U ngarkuan:", list(uploaded.keys()))


### Opsioni B — Mount Google Drive (rekomanduar për projekte)

In [ ]:
# nga google.colab import drive
# drive.mount('/content/drive')
# Pastaj kopjoni/ndryshoni rrugën sipas vendit ku i keni ruajtur PDF-të, p.sh.:
# !mkdir -p data/raw_pdfs
# !cp -n /content/drive/MyDrive/doctrine_sop_dataset/raw_pdfs/*.pdf data/raw_pdfs/


## 2) Ngarko benchmark-un (vendosni JSON-in këtu)

In [ ]:
# TODO: vendosni rrugën ku e ruani benchmark-un tuaj
BENCH_PATH = "data/qa/evaluation_benchmark_v2_complete.json"

if os.path.exists(BENCH_PATH):
    bench = json.load(open(BENCH_PATH, "r", encoding="utf-8"))
    print("Loaded benchmark:", len(bench))
else:
    bench = []
    print("Benchmark not found. Save it to", BENCH_PATH)


## 3) Auto-scoring (bazik)

In [ ]:
from rapidfuzz.distance import Levenshtein

def normalize(s):
    return re.sub(r'\s+', ' ', (s or '').strip().lower())

def em(a, b):
    return 1 if normalize(a) == normalize(b) else 0

def soft_match(a, b, max_dist=20):
    a2, b2 = normalize(a), normalize(b)
    if not a2 or not b2:
        return 0
    return 1 if Levenshtein.distance(a2, b2) <= max_dist else 0

# TODO: lidh me funksionin tuaj answer_with_rag / agent_answer
def system_answer(q):
    return {"answer": "TODO", "citations": []}

results = []
for item in bench:
    pred = system_answer(item["question"])
    expected = item["expected_answer"]
    is_refusal = item.get("expected_behavior") == "refusal"
    pred_is_refusal = "nuk" in normalize(pred["answer"]) and "dokument" in normalize(pred["answer"])
    results.append({
        "id": item["id"],
        "em": em(pred["answer"], expected),
        "soft": soft_match(pred["answer"], expected),
        "refusal_ok": int((not is_refusal) or (pred_is_refusal)),
        "has_citations": int(bool(pred.get("citations")))
    })

if results:
    print("N:", len(results))
    print("EM:", sum(r["em"] for r in results)/len(results))
    print("Soft:", sum(r["soft"] for r in results)/len(results))
    print("Refusal OK:", sum(r["refusal_ok"] for r in results)/len(results))
    print("Citations:", sum(r["has_citations"] for r in results)/len(results))


## 4) Dorëzim

- Tabela me rezultatet + 5 gabimet kryesore.
- Kategorizim: retrieval fail / prompt / refusal / citations.